In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Our universe
UNIVERSE = {
    'Quantum': ['IONQ', 'RGTI', 'QBTS', 'QUBT', 'ARQQ'],
    'Nuclear': ['UUUU', 'NXE', 'DNN', 'LEU', 'SMR'],
    'Space': ['RKLB', 'ASTS', 'LUNR', 'PL', 'BKSY'],
    'Semi': ['NVTS', 'WOLF', 'AEHR', 'ACLS', 'FORM'],
    'AI_Infra': ['SMCI', 'VRT', 'ANET', 'CRDO', 'COHR'],
}

# Flatten
ALL_TICKERS = [t for sector in UNIVERSE.values() for t in sector]
print(f"Scanning {len(ALL_TICKERS)} tickers for SILENCE...")

In [ ]:
# STEP 1: DEFINE WHAT "SILENCE" MEANS

silence_criteria = """
SILENCE = The opposite of what scanners look for

Traditional scanners find:
- High volume (excitement)
- Big price moves (momentum)
- Oversold/overbought (extremes)
- Breakouts (activity)

SILENCE SCANNER finds:
- Normal volume but TRENDING UP quietly
- Price FLAT (consolidation, not movement)
- RSI in the MIDDLE (40-60, no signal)
- No breakout, no breakdown (boring)
- BUT: Someone is accumulating (volume pattern)

THE HYPOTHESIS:
Silence + Accumulation = Pre-Setup
The "setup" gets manufactured LATER to bring retail in.
We want to be there BEFORE the setup appears.
"""

print(silence_criteria)

In [ ]:
# STEP 2: PULL DATA AND CALCULATE SILENCE METRICS

def calculate_silence_score(ticker, days=14):
    """Calculate how 'silent' a stock is - high score = quiet but accumulating"""
    try:
        # Get data
        end = datetime.now()
        start = end - timedelta(days=days+10)  # Extra buffer
        df = yf.download(ticker, start=start, end=end, progress=False)
        
        if len(df) < days:
            return None
        
        df = df.tail(days)
        
        # SILENCE METRICS:
        
        # 1. Price flatness (low = more silent)
        price_change = abs((df['Close'].iloc[-1] / df['Close'].iloc[0]) - 1) * 100
        price_flat_score = max(0, 10 - price_change)  # 10 if flat, 0 if moved 10%+
        
        # 2. Volume trend (positive = accumulation)
        vol_first_half = df['Volume'].iloc[:days//2].mean()
        vol_second_half = df['Volume'].iloc[days//2:].mean()
        vol_trend = (vol_second_half / vol_first_half - 1) * 100 if vol_first_half > 0 else 0
        vol_trend_score = min(10, max(0, vol_trend / 5))  # 10 if vol up 50%+
        
        # 3. RSI middle-ness (higher score if RSI 40-60)
        # Simple RSI calculation
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        current_rsi = rsi.iloc[-1] if not pd.isna(rsi.iloc[-1]) else 50
        
        # Score higher if RSI is in "boring" middle zone
        rsi_middle_score = 10 - abs(current_rsi - 50) / 5  # 10 if exactly 50, lower at extremes
        
        # 4. Volatility (low = silent)
        daily_returns = df['Close'].pct_change().dropna()
        volatility = daily_returns.std() * 100
        vol_score = max(0, 10 - volatility * 2)  # Lower volatility = higher score
        
        # COMPOSITE SILENCE SCORE
        # Weight: Price flatness most important, then vol trend (accumulation)
        silence_score = (
            price_flat_score * 0.3 +
            vol_trend_score * 0.4 +  # Accumulation is key
            rsi_middle_score * 0.15 +
            vol_score * 0.15
        )
        
        return {
            'ticker': ticker,
            'silence_score': round(silence_score, 2),
            'price_change_%': round(price_change, 2),
            'volume_trend_%': round(vol_trend, 2),
            'rsi': round(current_rsi, 1),
            'volatility_%': round(volatility, 2),
            'interpretation': 'QUIET + ACCUMULATING' if silence_score > 6 and vol_trend > 10 else 
                            'QUIET' if silence_score > 6 else
                            'ACTIVE' if silence_score < 4 else 'NEUTRAL'
        }
        
    except Exception as e:
        return None

print("Silence scoring function defined.")

In [ ]:
# STEP 3: SCAN ALL TICKERS

print("Scanning for SILENCE (this may take a moment)...")
print("Looking for: Price flat + Volume trending up + RSI middle")
print("=" * 60)

results = []
for ticker in ALL_TICKERS:
    result = calculate_silence_score(ticker)
    if result:
        results.append(result)
        status = "🔇" if result['interpretation'] == 'QUIET + ACCUMULATING' else \
                 "😴" if result['interpretation'] == 'QUIET' else \
                 "📢" if result['interpretation'] == 'ACTIVE' else "➖"
        print(f"{status} {ticker}: Score={result['silence_score']}, Vol Trend={result['volume_trend_%']:+.1f}%")

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('silence_score', ascending=False)

In [ ]:
# STEP 4: THE SILENT ACCUMULATORS

print("\n" + "=" * 60)
print("🔇 SILENT + ACCUMULATING (Pre-Setup Candidates)")
print("=" * 60)

silent_acc = results_df[
    (results_df['interpretation'] == 'QUIET + ACCUMULATING')
].head(10)

if len(silent_acc) > 0:
    for _, row in silent_acc.iterrows():
        print(f"\n🎯 {row['ticker']}")
        print(f"   Silence Score: {row['silence_score']}/10")
        print(f"   Price Change: {row['price_change_%']:+.1f}% (FLAT is good)")
        print(f"   Volume Trend: {row['volume_trend_%']:+.1f}% (UP is accumulation)")
        print(f"   RSI: {row['rsi']} (40-60 is 'no signal')")
        print(f"   → Someone buying while nobody watching")
else:
    print("No clear QUIET + ACCUMULATING patterns found.")
    print("Showing top 5 quietest:")
    for _, row in results_df.head(5).iterrows():
        print(f"   {row['ticker']}: Score={row['silence_score']}, Vol={row['volume_trend_%']:+.1f}%")

In [ ]:
# STEP 5: THE NOISY ONES (What everyone else is chasing)

print("\n" + "=" * 60)
print("📢 NOISY / ACTIVE (What everyone sees - probably late)")
print("=" * 60)

noisy = results_df[
    results_df['silence_score'] < 4
].sort_values('silence_score').head(5)

for _, row in noisy.iterrows():
    print(f"\n⚠️  {row['ticker']}")
    print(f"   Silence Score: {row['silence_score']}/10 (LOW = noisy)")
    print(f"   Price Change: {row['price_change_%']:+.1f}%")
    print(f"   RSI: {row['rsi']}")
    print(f"   → Everyone sees this. You're probably late.")

In [ ]:
# STEP 6: THE LEVEL 5 INSIGHT

level_5_insight = """
═══════════════════════════════════════════════════════════════
SILENCE SCANNER - LEVEL 5 INSIGHT
═══════════════════════════════════════════════════════════════

WHAT EVERY OTHER SCANNER DOES:
- Finds momentum (you're late)
- Finds oversold bounces (trap or opportunity - 50/50)
- Finds breakouts (already happening)
- Finds high volume (everyone sees it)

WHAT THIS SCANNER DOES:
- Finds SILENCE (nobody watching)
- Finds ACCUMULATION (someone IS watching, quietly)
- Finds PRE-SETUP (before the pattern gets manufactured)

THE HYPOTHESIS:
The stocks that run NEXT are the ones that are QUIET NOW
but have volume trending up (stealth accumulation).

The "setup" will be manufactured LATER:
- News will drop
- Technical pattern will "appear"
- Retail will pile in
- Pre-positioned holders will exit

THIS SCANNER TRIES TO SEE:
Where pre-positioning is happening BEFORE the setup is visible.

ACTION:
Track the QUIET + ACCUMULATING names for 1-2 weeks.
See if they later "suddenly" have news and run.
If pattern holds, we found the pre-setup signal.

═══════════════════════════════════════════════════════════════
"""

print(level_5_insight)

---
## 🐺 PROTOCOL COMPLETE

**What this notebook does differently:**
- Scans for ABSENCE of signals (not presence)
- Values boring, flat, quiet names
- Looks for hidden accumulation in the silence
- Inverts what every other scanner does

**The bet:**
The edge isn't in seeing the signal. Everyone sees it.
The edge is in seeing the silence before the signal appears.

**To validate:**
Track the QUIET + ACCUMULATING names for 2 weeks.
Do they later get "setups" and run?
If yes → we found the pre-setup indicator.